# Identifing Project Groupings 

Looking into the different agencies, Locodes and Project IDs to identify instances of multiple oblgiations for the same project or type of funding in the same timeframe or double entries.  

Will look into various geographical locations.

In [1]:
import pandas as pd
from siuba import *

import numpy as np
from sqlalchemy import create_engine
from siuba.sql import LazyTbl

In [2]:
from plotnine import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [3]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:.2f}".format

In [4]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/clean_obligated_waiting.csv', low_memory=False).drop('Unnamed: 0',axis=1)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [5]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
0,Obligated,BPMPL,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,5904,121,True
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,32D0,8,False
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,4820,4,False
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,5924,244,True
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,5924,214,True


In [6]:
## Number of Duplicates before dropping the columns

In [7]:
df.duplicated().sum()

0

In [8]:
df.isnull().sum()

location                      0
prefix                        0
project_no                    0
agency                        0
prepared_date                15
submit__to_hq_date          172
hq_review_date              203
submit_to_fhwa_date         221
to_fmis_date                221
fed_requested                 1
ac_requested                  0
total_requested               1
status_comment                0
locode                     1499
dist                          0
status                        0
dist_processing_days       1817
hq_processing_days          215
fhwa_processing_days        215
ftip_no                    4048
project_location            462
type_of_work                  0
seq                           0
date_request_initiated    10520
date_completed_request    10691
mpo                           1
warning                   20075
projectID                     0
projectNO                     0
compare_id_locode             0
dtype: int64

In [9]:
df.prefix.value_counts().sort_values(ascending=False)

HSIP        3318
CML         3293
STP         2786
ER          1590
BRLO        1402
            ... 
DPC            1
STIPL          1
HP21IMDL       1
DEMO17L        1
FTACRRS        1
Name: prefix, Length: 335, dtype: int64

In [10]:
df.agency.value_counts().sort_values(ascending=False).nlargest(10)

Humboldt County       627
Los Angeles           534
Fresno                459
Santa Cruz County     443
Los Angeles County    408
El Dorado County      297
Kern County           283
Fresno County         271
Bakersfield           270
Stockton              261
Name: agency, dtype: int64

In [11]:
df.agency.value_counts().sort_values(ascending=False).nsmallest(20)

Scotts Valley                                          1
Santa Cruz County Health Services Agency               1
Fthll/East Tca                                         1
Alameda County Transportation Improvement Authority    1
Coachella V A G                                        1
United States Forest Service                           1
Calaveras Council Of Governments                       1
Hawaiian Garden                                        1
Alameda County Waste Management Authority              1
Rollinghillsest                                        1
Samd                                                   1
Sjrt                                                   1
Corning                                                1
Union Pacific                                          1
Cap Soeast Con                                         1
Natomas Unified School District                        1
Marin County Transit District                          1
Trinidad                       

In [12]:
df>>count(_.mpo)>>arrange(-_.n)

,mpo,n
18,SCAG,4988
12,MTC,3236
13,NON-MPO,2592
14,SACOG,1865
2,CFCG,1404
8,KCOG,998
26,STANCOG,759
0,AMBAG,745
21,SJCG,515
29,TCAG,489


### Dropping Duplicates moved to the clean_all_sheets_v2.ipynb

code help: https://towardsdatascience.com/finding-and-removing-duplicate-rows-in-pandas-dataframe-c6117668631f

In [13]:
#duplicates in the data after removing some columns
df.duplicated().sum()

0

In [14]:
#count of obligations by dist
df>>count(_.dist)>>arrange(-_.n)

,dist,n
4,4,3248
6,6,3084
7,7,2787
3,3,2517
10,10,2203
5,5,1337
8,8,1309
1,1,1161
11,11,920
12,12,666


In [15]:
#of unique agencies
df >> group_by(_.dist) >> summarize(n = _.agency.nunique()) >> arrange(-_.n) 

,dist,n
4,4,154
7,7,118
10,10,71
8,8,69
3,3,65
6,6,55
12,12,40
5,5,38
11,11,32
2,2,22


## Where are there project groupings? 

Now what we do not have dupicates, we dive into the `project_location` and `type_of_work` columns to see if there are any occurances of agencies using multiple funding souces and projectIDs around the same timeframe

In [16]:
df = df.drop(['submit__to_hq_date',
              'hq_review_date',
              'submit_to_fhwa_date',
              'to_fmis_date',
              'status',
              'dist_processing_days',
              'hq_processing_days', 
              'fhwa_processing_days',
              'date_request_initiated',
              'date_completed_request',
              'warning'], axis=1)


In [17]:
cols = ['prepared_date']
df[cols] = df[cols].applymap(lambda x : pd.to_datetime(x, format = '%Y-%m-%d'))

### What dates have the most obligations? 

#### Adding year and month/year columns to the df

Adding these columns will allow us to branch outside the given day to identify obligations submitted around the same time

In [18]:
df.sample()

,location,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,projectID,projectNO,compare_id_locode
1617,Obligated,CML,5060(208),Fresno,2018-05-03,0.00,0.00,0.00,Authorized,5060,6,FRE110139,At The Sw Corner Of Shepherd And Willow Avenues,"Const Trailhead Impr, Park & Ride Parking Lot ...",5,CFCG,5060,208,True


In [19]:
df.loc[df["prepared_date"] == "NaT"]

,location,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,projectID,projectNO,compare_id_locode


In [20]:
df['prepared_date'].isnull().sum()

15

`Prepared Date` is the best to check when the obligations began. Ideally we would like to use the `Date Request Initiated` and `Date Completed Request`, but they have more NaT values

Since `prepared date` is the best, we will create columns containing the month and year of each obligation.

In [21]:
df['prepared_y_m'] = pd.to_datetime(df["prepared_date"].dt.strftime('%Y-%m'))

In [22]:
df['prepared_y'] = pd.to_datetime(df["prepared_date"].dt.strftime('%Y'))

In [23]:
df.sample()

,location,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,projectID,projectNO,compare_id_locode,prepared_y_m,prepared_y
15581,Obligated,HRRR,5916(110),Yuba County,2020-01-14,55080.26,0.00,61523.09,Authorized,5916,3,GROUP27,On Feather River Blvd Between County Rd 512 An...,Improve Horizontal Alignment (tc),5,SACOG,5916,110,True,2020-01-01,2020-01-01


### Filtering the dataframe using the new columns

In [24]:
df >> count(_.prepared_y) >> arrange(-_.n)

,prepared_y,n
8,2018-01-01,2827
4,2014-01-01,2724
9,2019-01-01,2665
5,2015-01-01,2575
6,2016-01-01,2505
10,2020-01-01,2350
7,2017-01-01,2333
11,2021-01-01,1917
3,2013-01-01,193
2,2012-01-01,7


In [25]:
df >> filter(_.prepared_y == '2018-01-01')

,location,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,projectID,projectNO,compare_id_locode,prepared_y_m,prepared_y
0,Obligated,BPMPL,5904(121),Humboldt County,2018-12-18,0.00,0.00,0.00,Authorized,5904,1,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NON-MPO,5904,121,True,2018-12-01,2018-01-01
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,11508.00,0.00,13000.00,Authorized,5910,1,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,NON-MPO,32D0,8,False,2018-12-01,2018-01-01
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,45499.64,0.00,51394.58,Authorized,5904,1,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,NON-MPO,4820,4,False,2018-12-01,2018-01-01
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,207002.00,0.00,247002.00,Authorized,5924,3,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,SACOG,5924,244,True,2018-12-01,2018-01-01
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,0.00,5680921.00,5702041.00,Authorized,5924,3,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,SACOG,5924,214,True,2018-12-01,2018-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19794,DISTRICT,SRTS,5006(681),Los Angeles,2018-03-05,829000.00,0.00,829000.00,..,5006,7,208,Ricardo Lizarraga E.s - Along San Pedro Street...,Construction Of Sidewalks And Crosswalks,2,SCAG,5006,681,True,2018-03-01,2018-01-01
19800,DISTRICT,BRLS,5954(108),San Bernardino County,2018-01-25,265590.00,0.00,300000.00,..,5954,8,200619,"Glen Helen Parkway Over Cajon Wash, Br. No. 54...",Bridge Replacement,5,SCAG,5954,108,True,2018-01-01,2018-01-01
19801,DISTRICT,SRTS,5420(023),Rancho Cucamonga,2018-08-23,90800.00,0.00,154000.00,..,5420,8,NaN,9th Street From 100' To 800' West Of Vinerayd ...,"Sidewalk, Curb And Gutter, Furnish And Install...",1,SCAG,5420,23,True,2018-08-01,2018-01-01
19813,DISTRICT,BRLS,5954(157),San Bernardino County,2018-01-04,2036190.00,0.00,2300000.00,..,5954,8,NaN,"Bridge No. 54c0127, Baker Bl Over Mojave River...",Bridge Replacement,1,SCAG,5954,157,True,2018-01-01,2018-01-01


In [26]:
df >> filter(_.prepared_y == '2014-01-01') >> count(_.agency) >> arrange(-_.n)

,agency,n
197,Los Angeles,110
127,Fresno,58
152,Humboldt County,51
25,Bakersfield,44
198,Los Angeles County,43
...,...,...
449,Western Shasta Resource Conservation District,1
452,Wildomar,1
454,Willows,1
464,Yuba City,1


In [27]:
(df
    >> filter(_.agency=='Humboldt County')
    >> count(_.prepared_y_m) 
    >> arrange(-_.n)
    >> filter(_.n >= 5)
)

,prepared_y_m,n
46,2018-03-01,50
48,2018-06-01,43
63,2019-09-01,30
55,2019-01-01,28
65,2019-11-01,26
5,2014-05-01,23
64,2019-10-01,22
81,2021-04-01,18
52,2018-10-01,17
49,2018-07-01,16


In [28]:
#using the day-month-year-format first
(df
    >> group_by(_.agency)
    >> count(_.prepared_date) 
    >> arrange(-_.n)
    >> filter(_.n >= 5)
)

,agency,prepared_date,n
4570,Humboldt County,2018-03-22,17
4571,Humboldt County,2018-03-29,16
1144,Calaveras County,2020-11-04,15
2541,Department Of Water Resources,2014-12-29,14
4461,Humboldt County,2014-05-22,13
...,...,...,...
13802,Santa Cruz County,2020-09-23,5
13804,Santa Cruz County,2020-09-28,5
14176,Shasta County,2021-05-18,5
14982,Stockton,2017-06-16,5


* 97 rows of agencies that have more than 5 obligations on the same date. Will get into Humboldt County in another notebook

In [29]:
#using the month-year date
(df
    >> group_by(_.prefix)
    >> count(_.prepared_y_m) 
    >> arrange(-_.n)
)

,prefix,prepared_y_m,n
1545,ER,2018-06-01,132
1562,ER,2019-11-01,101
1561,ER,2019-10-01,94
1033,CML,2014-07-01,73
2181,HSIP,2014-07-01,73
...,...,...,...
3442,VPPP,2018-03-01,1
3443,VPPP,2019-09-01,1
3444,VPPP,2019-11-01,1
3445,VPPTCS,2018-01-01,1


* 132 obligations in the same month of June 2018 for ER funds

In [30]:
#using the year date
(df
    >> group_by(_.prefix)
    >> count(_.prepared_y) 
    >> arrange(-_.n)
)

,prefix,prepared_y,n
612,HSIP,2019-01-01,537
247,CML,2014-01-01,469
248,CML,2015-01-01,449
425,ER,2018-01-01,446
251,CML,2018-01-01,443
...,...,...,...
1014,VPPP,2016-01-01,1
1015,VPPP,2017-01-01,1
1016,VPPP,2018-01-01,1
1018,VPPTCS,2018-01-01,1


* "HSIP" becomes the most obliagted by year, although it changes to "ER" when adding `agnecy` to the group_by column (below) 

In [31]:
#using the year date and adding agency
(df
    >> group_by(_.prefix, _.agency)
    >> count(_.prepared_y) 
    >> arrange(-_.n)
)

,prefix,agency,prepared_y,n
4100,ER,Humboldt County,2018-01-01,146
4101,ER,Humboldt County,2019-01-01,82
4270,ER,Santa Cruz County,2018-01-01,70
4271,ER,Santa Cruz County,2019-01-01,64
4102,ER,Humboldt County,2020-01-01,47
...,...,...,...,...
9281,VPPP,Santa Clara Valley Transportation Authority,2014-01-01,1
9282,VPPP,Santa Clara Valley Transportation Authority,2017-01-01,1
9283,VPPP,Southern California Association Of Governments,2015-01-01,1
9285,VPPTCS,"City & County Of San Francisco, Mta/Parking & ...",2018-01-01,1


### Agencies with 5 or more Obligations on a singular date

In [32]:
## grouping by project code too to see which if that is a factor... 
(df
    >> group_by(_.agency, _.project_no)
    >> count(_.prepared_date) 
    >> arrange(-_.n)
    >> filter(_.n >= 5)
)

,agency,project_no,prepared_date,n
1994,"City & County Of San Francisco, Mta/Parking & ...",6328(082),2016-07-20,10
10186,Napa County Transportation Planning Agency,6429(023),2018-04-05,10
8294,Los Angeles County Metropolitan Transportation...,6065(199),2015-07-15,9
8312,Los Angeles County Metropolitan Transportation...,6065(225),2018-05-31,9
2,Access Services,6312(022),2016-03-02,7
14034,San Diego Metropolitan Tranit System,7503(001),2020-07-03,7
19042,Ventura County Transportation Commission,6155(089),2015-11-16,6
8225,Los Angeles County,5953(721),2015-07-09,5
10695,Omnitrans,6046(011),2017-07-13,5
13385,Sacramento Regional Transit District,6005(067),2020-04-16,5


In [33]:
#will explore in `identifying_projects.ipynb`

In [34]:
(df
    >> group_by(_.agency, _.project_no)
    >> count(_.prepared_y_m) 
    >> arrange(-_.n)
    >> filter(_.n >= 5)
)

,agency,project_no,prepared_y_m,n
1976,"City & County Of San Francisco, Mta/Parking & ...",6328(082),2016-07-01,10
10100,Napa County Transportation Planning Agency,6429(023),2018-04-01,10
8228,Los Angeles County Metropolitan Transportation...,6065(199),2015-07-01,9
8245,Los Angeles County Metropolitan Transportation...,6065(225),2018-05-01,9
2,Access Services,6312(022),2016-03-01,7
13909,San Diego Metropolitan Tranit System,7503(001),2020-07-01,7
18874,Ventura County Transportation Commission,6155(089),2015-11-01,6
8159,Los Angeles County,5953(721),2015-07-01,5
10600,Omnitrans,6046(011),2017-07-01,5
13266,Sacramento Regional Transit District,6005(067),2020-04-01,5


* looks like the same number of entries. trying for the year. 

In [35]:
(df
    >> group_by(_.agency, _.project_no)
    >> count(_.prepared_y) 
    >> arrange(-_.n)
    >> filter(_.n >= 5)
)

,agency,project_no,prepared_y,n
1723,"City & County Of San Francisco, Mta/Parking & ...",6328(082),2016-01-01,10
8690,Napa County Transportation Planning Agency,6429(023),2018-01-01,10
7103,Los Angeles County Metropolitan Transportation...,6065(199),2015-01-01,9
7119,Los Angeles County Metropolitan Transportation...,6065(225),2018-01-01,9
2,Access Services,6312(022),2016-01-01,7
11992,San Diego Metropolitan Tranit System,7503(001),2020-01-01,7
16261,Ventura County Transportation Commission,6155(089),2015-01-01,6
808,Brawley,5167(037),2018-01-01,5
1172,Calipatria,5243(002),2016-01-01,5
2007,Compton,5078(012),2018-01-01,5


* got a few more entries! 


### Project location

In [36]:
(df
    >> count(_.project_location) 
    >> arrange(-_.n)
    >> filter(_.n >= 10)
)

,project_location,n
6925,San Francisco Bay Area,65
8196,Various Locations,31
6846,Sacog Region,29
3028,Golden Gate Bridge,27
7864,Throughout Kern County,22
1830,Citywide,18
8205,Various Locations (see Comments),14
8262,Various Locations In San Diego County,14
2030,County Of Madera,13
2623,FTA transfer,13


In [37]:
## seeing which agencies use vauge language 
(df
    >> filter(_.project_location == ("Various Locations"))
    >> group_by(_.agency)
    >> count(_.prefix)
)

,agency,prefix,n
0,Alameda County,CML,4
1,Fresno,ATPLNI,4
2,Richmond,SRTS,4
3,San Francisco County Transportation Authority,CML,1
4,San Francisco Department Of Public Health,CML,1
5,Santa Clara County,BHLO,1
6,Sonoma County Transportation Authority,CMLNI,3
7,Stanislaus County,HRRR,2
8,Taft,CML,3
9,Tehama County,BPMPL,2


In [38]:
(df
    >> filter(_.project_location == ("Various"))
    >> group_by(_.agency)
    >> count(_.prefix)
)

,agency,prefix,n
0,Butte County,BPMPL,2
1,Calabasas,SRTS,2
2,Delano,SRTS,2
3,San Marcos,HSIP,4
4,Siskiyou County,BRLS,2


In [39]:
(df
    >> filter(_.project_location == ("Citywide"))
    >> group_by(_.agency)
    >> count(_.prefix)
)

,agency,prefix,n
0,Bakersfield,BPMPL,3
1,Compton,HP21L,5
2,Los Angeles,ESPL,2
3,Oakland,CMLNI,3
4,San Jose,CML,1
5,San Mateo,CMLNI,3
6,Santa Clarita,CML,1


In [40]:
(df
    >> filter(_.project_location == ("Various Locations (see Comments)"))
    >> group_by(_.agency)
    >> count(_.prefix)
)

,agency,prefix,n
0,Imperial,CML,1
1,Imperial County,CML,3
2,Placer County,CML,1
3,Placer County,HSIP,7
4,Placer County,STP,2


### Type of Work

In [41]:
df.type_of_work.nunique()

6882

In [42]:
df >> count(_.type_of_work) >> arrange(-_.n) >> filter(_.n>10)

,type_of_work,n
592,Bridge Replacement (tc),679
582,Bridge Replacement,519
2014,FTA Transfer,462
3979,Permanent Restoration,282
5289,Road Rehabilitation,269
...,...,...
5673,Seismic Retrofit (tc),11
5745,Shoulder Widening,11
5964,Stabilize Embankment And Reconstruct Roadway,11
6054,Streetscape,11


In [43]:
(df >> count(_.type_of_work) 
    >> summarize(med = _.n.median()))

,med
0,2.00


In [44]:
(df
    >> group_by(_.type_of_work)
    >> count(_.prefix) 
    >> arrange(-_.n)
    >> filter(_.n>20)
)

,type_of_work,prefix,n
659,Bridge Replacement (tc),BRLO,646
639,Bridge Replacement,BRLS,413
2222,FTA Transfer,FTACML,266
5871,Road Rehabilitation,STP,256
4362,Permanent Restoration,ER,195
531,Bridge Preventive Maintenance,BPMPL,160
1930,Emergency Opening,ER,134
4040,Pavement Rehabilitation,STP,111
6682,Storm Damage Restoration,ER,102
2237,FTA Transfer,FTASTPL,94


In [45]:
(df
    >> group_by(_.type_of_work, _.agency)
    >> count(_.prefix) 
    >> arrange(-_.n)
    >> filter(_.n>20)
)

,type_of_work,agency,prefix,n
5157,Permanent Restoration,Humboldt County,ER,134
7851,Storm Damage Restoration,Humboldt County,ER,97
86,Ac Overlay (tc),Fresno,STP,69
5167,Permanent Restoration,Mendocino County,ER,47
962,Bridge Replacement (tc),El Dorado County,BRLO,45
5155,Permanent Restoration,Humboldt County,ACST-ER,44
2734,FTA Transfer,Los Angeles County Metropolitan Transportation...,FTACML,38
6270,Replace 2 Lane Bridge With 2 Lane Bridge (tc),Fresno County,BRLO,37
6273,Replace 2 Lane Bridge With 2 Lane Bridge (tc),Tulare County,BRLO,35
1032,Bridge Replacement (tc),Yuba County,BRLO,29


#### Adding year to type_of_work

In [46]:
(df
    >> group_by(_.agency, _.type_of_work, _.prefix)
    >> count(_.prepared_y) 
    >> arrange(-_.n)
    >> filter(_.n >= 10)
)

,agency,type_of_work,prefix,prepared_y,n
4302,Humboldt County,Storm Damage Restoration,ER,2018-01-01,84
4215,Humboldt County,Permanent Restoration,ER,2019-01-01,47
4214,Humboldt County,Permanent Restoration,ER,2018-01-01,36
4203,Humboldt County,Permanent Restoration,ACST-ER,2019-01-01,27
10732,San Bernardino County,Replace Two Lane Timber Bridge With Two Lane B...,BRLS,2021-01-01,18
6903,Mendocino County,Permanent Restoration,ER,2018-01-01,17
3170,Fresno,Ac Overlay (tc),STP,2016-01-01,16
3173,Fresno,Ac Overlay (tc),STP,2019-01-01,16
4216,Humboldt County,Permanent Restoration,ER,2020-01-01,16
4217,Humboldt County,Permanent Restoration,ER,2021-01-01,16
